In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from numba import njit, prange
from scipy import interpolate

In [ ]:
kbT = 0.4

D = 6
H = 8 * D
L = 50 * D
n = 10

# H = 30
# L = 50
# n = 35

# v_max = np.sqrt(2*kbT) / 4
v_max = 0.2

N = n * L * H
r = np.zeros((N, 2), dtype=np.float32)
v = np.random.normal(0, np.sqrt(kbT), (N, 2)).astype(np.float32)

y0 = (H-D)/2
x0 = L/4 - D/2

for i in range(L):
    for j in range(H):
        if (i >= x0) and (i < x0+D) and (j >= y0) and (j < y0+D):
            continue
        r[(i*H+j)*n:(i*H+j+1)*n, :] = np.random.rand(n, 2) + np.array([i, j])

intake = r[:, 0] <= 10
v[intake, 0] += 4*v_max * (H - r[intake, 1]) * r[intake, 1] / H**2

In [ ]:
R_plus = np.array([[0., -1.], [1., 0.]], dtype=np.float32)
R_minus = np.array([[0., 1.], [-1., 0.]], dtype=np.float32)

In [ ]:
@njit(parallel=True, fastmath=True)
def update(r, v):
    r += v

    # pbc on x
    r[r[:, 0] > L, 0] = r[r[:, 0] > L, 0] - L
    r[r[:, 0] <= 0, 0] = r[r[:, 0] <= 0, 0] + L
    
    # bounce-back rule for walls
    to_bounce = r[:, 1] <= 0
    tac = r[to_bounce, 1] / v[to_bounce, 1]
    r[to_bounce, 0] = r[to_bounce, 0] - 2*v[to_bounce, 0]*tac
    r[to_bounce, 1] = r[to_bounce, 1] - 2*v[to_bounce, 1]*tac
    v[to_bounce] = -v[to_bounce]
    
    to_bounce = r[:, 1] > H
    tac = (r[to_bounce, 1] - H) / v[to_bounce, 1]
    r[to_bounce, 0] = r[to_bounce, 0] - 2*v[to_bounce, 0]*tac
    r[to_bounce, 1] = r[to_bounce, 1] - 2*v[to_bounce, 1]*tac
    v[to_bounce] = -v[to_bounce]

    # bounce-back rule for cilinder
    in_D = (r[:, 1] > y0) & (r[:, 1] < (y0+D)) & (r[:, 0] > x0) & (r[:, 0] < (x0+D))

    t_x0 = (r[in_D, 0] - x0) / v[in_D, 0]
    t_y0 = (r[in_D, 1] - y0) / v[in_D, 1]
    t_x0D = (r[in_D, 0] - (x0+D)) / v[in_D, 0]
    t_y0D = (r[in_D, 1] - (y0+D)) / v[in_D, 1]

    in_x0 = (t_x0 < 1) & (t_x0 > 0)
    in_y0 = (t_y0 < 1) & (t_y0 > 0)
    t_x0[in_x0 & in_y0] = np.where(t_x0[in_x0 & in_y0] > t_y0[in_x0 & in_y0], 2, t_x0[in_x0 & in_y0])
    t_y0[in_x0 & in_y0] = np.where(t_x0[in_x0 & in_y0] < t_y0[in_x0 & in_y0], 2, t_y0[in_x0 & in_y0])
    
    in_x0 = (t_x0 < 1) & (t_x0 > 0)
    in_y0D = (t_y0D < 1) & (t_y0D > 0)
    t_x0[in_x0 & in_y0D] = np.where(t_x0[in_x0 & in_y0D] > t_y0D[in_x0 & in_y0D], 2, t_x0[in_x0 & in_y0D])
    t_y0D[in_x0 & in_y0D] = np.where(t_x0[in_x0 & in_y0D] < t_y0D[in_x0 & in_y0D], 2, t_y0D[in_x0 & in_y0D])

    in_x0D = (t_x0D < 1) & (t_x0D > 0)
    in_y0D = (t_y0D < 1) & (t_y0D > 0)
    t_x0D[in_x0D & in_y0D] = np.where(t_x0D[in_x0D & in_y0D] > t_y0D[in_x0D & in_y0D], 2, t_x0D[in_x0D & in_y0D])
    t_y0D[in_x0D & in_y0D] = np.where(t_x0D[in_x0D & in_y0D] < t_y0D[in_x0D & in_y0D], 2, t_y0D[in_x0D & in_y0D])

    in_x0D = (t_x0D < 1) & (t_x0D > 0)
    in_y0 = (t_y0 < 1) & (t_y0 > 0)
    t_x0D[in_x0D & in_y0] = np.where(t_x0D[in_x0D & in_y0] > t_y0[in_x0D & in_y0], 2, t_x0D[in_x0D & in_y0])
    t_y0[in_x0D & in_y0] = np.where(t_x0D[in_x0D & in_y0] < t_y0[in_x0D & in_y0], 2, t_y0[in_x0D & in_y0])

    r[in_D, 0] = np.where((t_x0 < 1) & (t_x0 > 0), r[in_D, 0] - 2*t_x0*v[in_D, 0], r[in_D, 0])
    r[in_D, 1] = np.where((t_x0 < 1) & (t_x0 > 0), r[in_D, 1] - 2*t_x0*v[in_D, 1], r[in_D, 1])

    r[in_D, 0] = np.where((t_y0 < 1) & (t_y0 > 0), r[in_D, 0] - 2*t_y0*v[in_D, 0], r[in_D, 0])
    r[in_D, 1] = np.where((t_y0 < 1) & (t_y0 > 0), r[in_D, 1] - 2*t_y0*v[in_D, 1], r[in_D, 1])

    r[in_D, 0] = np.where((t_x0D < 1) & (t_x0D > 0), r[in_D, 0] - 2*t_x0D*v[in_D, 0], r[in_D, 0])
    r[in_D, 1] = np.where((t_x0D < 1) & (t_x0D > 0), r[in_D, 1] - 2*t_x0D*v[in_D, 1], r[in_D, 1])

    r[in_D, 0] = np.where((t_y0D < 1) & (t_y0D > 0), r[in_D, 0] - 2*t_y0D*v[in_D, 0], r[in_D, 0])
    r[in_D, 1] = np.where((t_y0D < 1) & (t_y0D > 0), r[in_D, 1] - 2*t_y0D*v[in_D, 1], r[in_D, 1])

    in_x0 = (t_x0 < 1) & (t_x0 > 0)
    in_x0D = (t_x0D < 1) & (t_x0D > 0)
    in_y0 = (t_y0 < 1) & (t_y0 > 0)
    in_y0D = (t_y0D < 1) & (t_y0D > 0)

    # bounce-back rule for cilinder
    t_x0 = (r[:, 0] - x0) / v[:, 0]
    t_y0 = (r[:, 1] - y0) / v[:, 1]
    t_x0D = (r[:, 0] - (x0+D)) / v[:, 0]
    t_y0D = (r[:, 1] - (y0+D)) / v[:, 1]
    
    in_x0 = (t_x0 < 1) & (t_x0 > 0)
    y_in_x0 = r[:, 1] - v[:, 1] * t_x0
    in_x0 = in_x0 & (y_in_x0 < (y0+D)) & (y_in_x0 > y0)
    
    in_x0D = (t_x0D < 1) & (t_x0D > 0)
    y_in_x0D = r[:, 1] - v[:, 1] * t_x0D
    in_x0D = in_x0D & (y_in_x0D < (y0+D)) & (y_in_x0D > y0)
    
    in_y0 = (t_y0 < 1) & (t_y0 > 0)
    x_in_y0 = r[:, 0] - v[:, 0] * t_y0
    in_y0 = in_y0 & (x_in_y0 < (x0+D)) & (x_in_y0 > x0)
    
    in_y0D = (t_y0D < 1) & (t_y0D > 0)
    x_in_y0D = r[:, 0] - v[:, 0] * t_y0D
    in_y0D = in_y0D & (x_in_y0D < (x0+D)) & (x_in_y0D > x0)
    
    both = (in_x0 & in_y0)
    in_x0[both] = t_x0[both] > t_y0[both]
    in_y0[both] = t_x0[both] < t_y0[both]
    
    both = (in_x0 & in_y0D)
    in_x0[both] = t_x0[both] > t_y0D[both]
    in_y0D[both] = t_x0[both] < t_y0D[both]
    
    both = (in_x0D & in_y0D)
    in_x0D[both] = t_x0D[both] > t_y0D[both]
    in_y0D[both] = t_x0D[both] < t_y0D[both]
    
    both = (in_x0D & in_y0)
    in_x0D[both] = t_x0D[both] > t_y0[both]
    in_y0[both] = t_x0D[both] < t_y0[both]
    
    r[in_x0, 0] = r[in_x0, 0] - 2*v[in_x0, 0]*t_x0[in_x0]
    r[in_x0, 1] = r[in_x0, 1] - 2*v[in_x0, 1]*t_x0[in_x0]
    
    r[in_y0, 0] = r[in_y0, 0] - 2*v[in_y0, 0]*t_y0[in_y0]
    r[in_y0, 1] = r[in_y0, 1] - 2*v[in_y0, 1]*t_y0[in_y0]
    
    r[in_x0D, 0] = r[in_x0D, 0] - 2*v[in_x0D, 0]*t_x0D[in_x0D]
    r[in_x0D, 1] = r[in_x0D, 1] - 2*v[in_x0D, 1]*t_x0D[in_x0D]
    
    r[in_y0D, 0] = r[in_y0D, 0] - 2*v[in_y0D, 0]*t_y0D[in_y0D]
    r[in_y0D, 1] = r[in_y0D, 1] - 2*v[in_y0D, 1]*t_y0D[in_y0D]

    f_par = 2*np.sum(v[in_x0, 0]) + 2*np.sum(v[in_x0D, 0])

    v[in_x0, :] = -v[in_x0, :]
    v[in_y0, :] = -v[in_y0, :]
    v[in_x0D, :] = -v[in_x0D, :]
    v[in_y0D, :] = -v[in_y0D, :]

    eps = np.random.rand(L*H)
    sf = np.random.random(4) - 0.5
    for i in prange(L):
        # first shifted cell
        cell = (r[:, 0] > (i+sf[0])) & (r[:, 0] <= (i+sf[1]+1)) & (r[:, 1] > sf[2]) & (r[:, 1] <= (1+sf[3]))
        v_cell  = v[cell, :]

        if (n - v_cell.shape[0]) > 0:
            a = np.random.normal(0, np.sqrt((n - v_cell.shape[0])*kbT), 2).astype(np.float32)
            u = (np.sum(v_cell, axis=0) + a) / n
        else:
            u = np.sum(v_cell, axis=0) / v_cell.shape[0]

        if eps[i*H] > 0.5:
            v[cell, :] = u + np.dot(v_cell-u, R_plus)  # should transpose but symmetry
        else:
            v[cell, :] = u + np.dot(v_cell-u, R_minus) # should transpose but symmetry

        # bulk cells
        for j in range(1, H-1):
            if (i >= x0) and (i < x0+D) and (j >= y0) and (j < y0+D):
                continue

            # x shift and y shift
            cell = (r[:, 0] > (i+sf[0])) & (r[:, 0] <= (i+sf[1]+1)) & (r[:, 1] > (j+sf[2])) & (r[:, 1] <= (j+sf[3]+1))
            v_cell  = v[cell, :]

            if (((i==(x0-1)) and (j >= y0) and (j < (y0+D))) or \
                ((i==(x0+D)) and (j >= y0) and (j < (y0+D))) or \
                ((j==(y0-1)) and (i >= x0) and (i < (x0+D))) or \
                ((j==(y0+D)) and (i >= x0) and (i < (x0+D)))) and (n - v_cell.shape[0]) > 0:
                a = np.random.normal(0, np.sqrt((n - v_cell.shape[0])*kbT), 2).astype(np.float32)
                u = (np.sum(v_cell, axis=0) + a) / n
            else:
                u = np.sum(v_cell, axis=0) / v_cell.shape[0]

            if eps[i*H+j+1] > 0.5:
                v[cell, :] = u + np.dot(v_cell-u, R_plus)  # should transpose but symmetry
            else:
                v[cell, :] = u + np.dot(v_cell-u, R_minus) # should transpose but symmetry

        # last shifted cell
        cell = (r[:, 0] > (i+sf[0])) & (r[:, 0] <= (i+1+sf[1])) & (r[:, 1] > (H-1+sf[2])) & (r[:, 1] <= (H+sf[3]))
        v_cell  = v[cell, :]

        if (n - v_cell.shape[0]) > 0:
            a = np.random.normal(0, np.sqrt((n - v_cell.shape[0])*kbT), 2).astype(np.float32)
            u = (np.sum(v_cell, axis=0) + a) / n
        else:
            u = np.sum(v_cell, axis=0) / v_cell.shape[0]

        if eps[(i+1)*H-1] > 0.5:
            v[cell, :] = u + np.dot(v_cell-u, R_plus)  # should transpose but symmetry
        else:
            v[cell, :] = u + np.dot(v_cell-u, R_minus) # should transpose but symmetry
    
    intake = r[:, 0] <= 10
    v[intake, 0] = np.random.normal(0, np.sqrt(kbT), np.sum(intake))
    v[intake, 0] += 4*v_max * (H - r[intake, 1]) * r[intake, 1] / H**2
    return r, v, f_par

In [ ]:
r, v, f = update(r, v)
%timeit update(r, v)

In [ ]:
M = 2000
M_eq = 1000

columns = np.arange(H) + 0.5

# ux = np.zeros((H, M-M_eq))
F = np.zeros(M-M_eq)
# vx = np.zeros(M+1)
# vy = np.zeros(M+1)

n_probes = 31
v_rec = np.zeros((M-M_eq, n_probes))

for i in range(M+1):
    r, v, f = update(r, v)

    # for j in range(H):
    #     column = (j < r[:, 1]) & (r[:, 1] <= j+1)
    #     if i > M_eq:
            # ux[j, i-1-M_eq] += np.mean(v[column, 0])
            # F[i-1-M_eq] = f
    if i > M_eq:
        F[i-1-M_eq] = f

    # vx[i] = np.mean(v[:, 0])
    # vy[i] = np.mean(v[:, 1])

# d_ux = np.std(ux, axis=1) / np.sqrt(M-M_eq)
# ux = np.mean(ux, axis=1)

# plt.errorbar(columns, ux, yerr=d_ux, ms=3, fmt=".")

# plt.plot(columns, 4*v_max * (H - columns) * columns / H**2)
# plt.legend()
# plt.grid()

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
uxy = lambda y, v: 4*v * (H - y) * y / H**2

popt, perr = curve_fit(uxy, columns, ux, sigma=d_ux)

v_max = popt[0]